In [1]:
import os, sys, json
sys.path.append(os.path.realpath(f"{os.getcwd()}/../../"))

from autocog.pm.sta.backend import Backend
from autocog.pm.sta.syntax import Syntax
from autocog.pm.sta.automaton import Automaton as STA
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
source = """
format bool {
    is enum("true","false");
}
prompt mmlu_main {
    is {
        A is bool;
        B[0:2] is {
            C is bool;
            D[2:4] is bool;
        }
        E[0:2] is {
          F is bool;
        }
        G is {
          H is bool;
          I is bool;
        }
        J is bool;
    }
    channel {
        to .B.C from ?inC;
        to .E   from ?inE;
        to .G   from ?inG;
    }
}"""
samples = [
    {
        "inC" : [ 'true', 'false' ],
        "inE" : [ { 'F' : 'true' } ],
        "inG" : { 'H' : 'true' }
    }
]
syntax = Syntax(header_pre_post=('[INST]\n<<SYS>>You are an expert interacting with the world using an interactive prompting system.<</SYS>>','[/INST]'))
backend = Backend(source=source)
for (name, prompt) in backend.program.prompts.items():
    sta = STA(prompt=prompt)

    sta.build_abstract()
    # display(wrap_graphviz(sta.toGraphViz_abstract()))

    sta.build_concrete()
    # display(wrap_graphviz(sta.toGraphViz_concrete()))

    for sample in samples:
        fta = sta.instantiate(syntax, data=sample)
        dotstr = fta.toGraphViz()
        display(wrap_graphviz(dotstr))